**(1) Determine Intersection Point using CCW**

In [ ]:
import tkinter as tk
import matplotlib.pyplot as plt
import numpy as np

class LineDrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Line Drawing Input")

        self.canvas = tk.Canvas(root, width=500, height=500, bg='white')
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.points = []
        self.lines = []
        self.line1_coords = []
        self.line2_coords = []

        self.canvas.bind("<Button-1>", self.on_click)
        self.submit_button = tk.Button(root, text="Submit", command=self.submit_and_close)
        self.submit_button.pack()

    def on_click(self, event):
        if len(self.lines) == 2:
            return  # Allow only two lines to be drawn

        x, y = event.x, event.y
        point = (x, y)
        self.points.append(point)

        self.draw_scatter_point(point)

        if len(self.points) == 4:
            self.draw_line_segments()

    def draw_scatter_point(self, point):
        x, y = point
        radius = 5  # Adjust the radius of the scatter points as needed
        self.canvas.create_oval(x - radius, y - radius, x + radius, y + radius, fill='green')

    def draw_line_segments(self):
        line1 = self.canvas.create_line(self.points[:2], fill='blue')
        line2 = self.canvas.create_line(self.points[2:], fill='red')

        self.lines = [line1, line2]
        self.line1_coords = [(self.points[0][0], self.points[0][1]), (self.points[1][0], self.points[1][1])]
        self.line2_coords = [(self.points[2][0], self.points[2][1]), (self.points[3][0], self.points[3][1])]

    def submit_and_close(self):
        if len(self.lines) == 2:
            print("Line 1 Coordinates:", self.line1_coords)
            print("Line 2 Coordinates:", self.line2_coords)

            self.root.destroy()
        else:
            print("Please draw exactly two lines.")

# Function to check if points are counterclockwise
def ccw(A, B, C):
    return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

# Function to check if lines are collinear
def collinear(A, B, C, D):
    return (C[1] - A[1]) * (B[0] - A[0]) == (B[1] - A[1]) * (C[0] - A[0]) and \
           (D[1] - A[1]) * (B[0] - A[0]) == (B[1] - A[1]) * (D[0] - A[0])

# Function to check if lines intersect
def intersect(A, B, C, D):
    return ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D)

# Function to calculate the angle between two lines
def angle_between_lines(line1, line2):
    v1 = np.array([line1[1][0] - line1[0][0], line1[1][1] - line1[0][1]])
    v2 = np.array([line2[1][0] - line2[0][0], line2[1][1] - line2[0][1]])
    unit_v1 = v1 / np.linalg.norm(v1)
    unit_v2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_v1, unit_v2)
    angle = np.arccos(dot_product)
    return np.degrees(angle)

if __name__ == "__main__":
    root = tk.Tk()
    app = LineDrawingApp(root)
    root.mainloop()

    # After closing the app, use the coordinates to check if lines intersect
    line1_coords = app.line1_coords
    line2_coords = app.line2_coords

    # Plot lines
    fig, ax = plt.subplots()
    ax.set_xlim(0, 500)
    ax.set_ylim(0, 500)

    l1, = ax.plot([line1_coords[0][0], line1_coords[1][0]], [line1_coords[0][1], line1_coords[1][1]], label='Line 1')
    l2, = ax.plot([line2_coords[0][0], line2_coords[1][0]], [line2_coords[0][1], line2_coords[1][1]], label='Line 2')

    # Check if lines intersect
    result_text = ""
    if collinear(line1_coords[0], line1_coords[1], line2_coords[0], line2_coords[1]):
        result_text = "\n\nLines are collinear."
        # Show a message in the plot
        ax.text(0.5, 0.9, result_text, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    elif intersect(line1_coords[0], line1_coords[1], line2_coords[0], line2_coords[1]):
        result_text = "\n\nLines intersect."
    else:
        result_text = "\n\nLines do not intersect."

    # Check for counterclockwise orientation
    if ccw(line1_coords[0], line1_coords[1], line2_coords[1]):
        result_text += " Line2 is CCW to Line1."
        # Draw a dotted line indicating the counterclockwise angle
        dotted_line = [(line1_coords[0][0], line2_coords[1][0]), (line1_coords[0][1], line2_coords[1][1])]
        ax.plot(dotted_line[0], dotted_line[1], linestyle='dotted', color='red', label='CCW Angle')
    else:
        result_text += " Line2 is not CCW to Line1."

    # Display result
    result_text += f'\nAngle between lines: {angle_between_lines(line1_coords, line2_coords):.2f}°'

    ax.text(0.5, -0.1, result_text, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

    plt.legend()
    plt.show()

    print(f"Time Complexity  : O({1})")
    print(f"Space Complexity : O({1})")

**(2) Determine Intersection Point using Bounding Boxes**

In [ ]:
import tkinter as tk
import matplotlib.pyplot as plt

class LineDrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Line Drawing Input")

        self.canvas = tk.Canvas(root, width=500, height=500, bg='white')
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.points = []
        self.lines = []
        self.line1_coords = []
        self.line2_coords = []

        self.canvas.bind("<Button-1>", self.on_click)
        self.submit_button = tk.Button(root, text="Submit", command=self.submit_and_close)
        self.submit_button.pack()

    def on_click(self, event):
        if len(self.lines) == 2:
            return  # Allow only two lines to be drawn

        x, y = event.x, event.y
        point = (x, y)
        self.points.append(point)

        self.draw_scatter_point(point)

        if len(self.points) == 4:
            self.draw_line_segments()

    def draw_scatter_point(self, point):
        x, y = point
        radius = 5  # Adjust the radius of the scatter points as needed
        self.canvas.create_oval(x - radius, y - radius, x + radius, y + radius, fill='green')

    def draw_line_segments(self):
        line1 = self.canvas.create_line(self.points[:2], fill='blue')
        line2 = self.canvas.create_line(self.points[2:], fill='red')

        self.lines = [line1, line2]
        self.line1_coords = [(self.points[0][0], self.points[0][1]), (self.points[1][0], self.points[1][1])]
        self.line2_coords = [(self.points[2][0], self.points[2][1]), (self.points[3][0], self.points[3][1])]

    def submit_and_close(self):
        if len(self.lines) == 2:
            print("Line 1 Coordinates:", self.line1_coords)
            print("Line 2 Coordinates:", self.line2_coords)
            # Check if lines intersect
            intersection = None
            if bounding_box_check(self.line1_coords, self.line2_coords):
                intersection = line_intersection(self.line1_coords, self.line2_coords)

            # Plot lines and points
            self.plot_lines_and_points()

            # Close the input app
            self.root.destroy()

            # Display result below the plot
            self.display_result(intersection)

        else:
            print("Please draw exactly two lines.")

    def plot_lines_and_points(self):
        # Plot lines
        plt.figure()
        plt.plot([self.line1_coords[0][0], self.line1_coords[1][0]], [self.line1_coords[0][1], self.line1_coords[1][1]], label='Line 1')
        plt.plot([self.line2_coords[0][0], self.line2_coords[1][0]], [self.line2_coords[0][1], self.line2_coords[1][1]], label='Line 2')

        # Plot points
        plt.scatter(*zip(*self.points), color='red', label='Points')

        # Plot bounding boxes with dotted lines
        self.plot_bounding_boxes(self.line1_coords, 'blue')
        self.plot_bounding_boxes(self.line2_coords, 'green')

        plt.legend()
        plt.show()

    def plot_bounding_boxes(self, line_coords, color):
        box_x = sorted([line_coords[0][0], line_coords[1][0]])
        box_y = sorted([line_coords[0][1], line_coords[1][1]])

        plt.plot([box_x[0], box_x[0]], [box_y[0], box_y[1]], linestyle='dotted', color=color)
        plt.plot([box_x[1], box_x[1]], [box_y[0], box_y[1]], linestyle='dotted', color=color)
        plt.plot([box_x[0], box_x[1]], [box_y[0], box_y[0]], linestyle='dotted', color=color)
        plt.plot([box_x[0], box_x[1]], [box_y[1], box_y[1]], linestyle='dotted', color=color)

    def display_result(self, intersection):
        result_text = "Intersection Point: " + str(intersection) if intersection else "No Intersection"
        print(result_text)

def bounding_box_check(line1, line2):
    """ Check if the bounding boxes of the line segments overlap """
    box1_x = sorted([line1[0][0], line1[1][0]])
    box1_y = sorted([line1[0][1], line1[1][1]])
    box2_x = sorted([line2[0][0], line2[1][0]])
    box2_y = sorted([line2[0][1], line2[1][1]])

    return not (box1_x[1] < box2_x[0] or box1_x[0] > box2_x[1] or
                box1_y[1] < box2_y[0] or box1_y[0] > box2_y[1])

def line_intersection(line1, line2):
    """ Find intersection point of two line segments (if exists) """
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None  # Lines don't intersect

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    if (min(line1[0][0], line1[1][0]) <= x <= max(line1[0][0], line1[1][0]) and
        min(line1[0][1], line1[1][1]) <= y <= max(line1[0][1], line1[1][1]) and
        min(line2[0][0], line2[1][0]) <= x <= max(line2[0][0], line2[1][0]) and
        min(line2[0][1], line2[1][1]) <= y <= max(line2[0][1], line2[1][1])):
        return (x, y)
    else:
        return None  # Intersection is not within the line segments

if __name__ == "__main__":
    root = tk.Tk()
    app = LineDrawingApp(root)
    root.mainloop()

    print(f"\nTime Complexity  : O({1})")
    print(f"Space Complexity : O({1})")

**(3) Determine Intersection Point using Cramer's Rule**

In [ ]:
import tkinter as tk
import matplotlib.pyplot as plt
import numpy as np
import time

class LineDrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Line Drawing Input")

        self.canvas = tk.Canvas(root, width=500, height=500, bg='white')
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.points = []
        self.lines = []
        self.line1_coords = []
        self.line2_coords = []

        self.canvas.bind("<Button-1>", self.on_click)
        self.submit_button = tk.Button(root, text="Submit", command=self.submit_and_close)
        self.submit_button.pack()

    def on_click(self, event):
        if len(self.lines) == 2:
            return  # Allow only two lines to be drawn

        x, y = event.x, event.y
        point = (x, y)
        self.points.append(point)

        self.draw_scatter_point(point)

        if len(self.points) == 4:
            self.draw_line_segments()

    def draw_scatter_point(self, point):
        x, y = point
        radius = 5  # Adjust the radius of the scatter points as needed
        self.canvas.create_oval(x - radius, y - radius, x + radius, y + radius, fill='green')

    def draw_line_segments(self):
        line1 = self.canvas.create_line(self.points[:2], fill='blue')
        line2 = self.canvas.create_line(self.points[2:], fill='red')

        self.lines = [line1, line2]
        self.line1_coords = [(self.points[0][0], self.points[0][1]), (self.points[1][0], self.points[1][1])]
        self.line2_coords = [(self.points[2][0], self.points[2][1]), (self.points[3][0], self.points[3][1])]

    def submit_and_close(self):
        if len(self.lines) == 2:
            print("Line 1 Coordinates:", self.line1_coords)
            print("Line 2 Coordinates:", self.line2_coords)

            self.root.destroy()
        else:
            print("Please draw exactly two lines.")

def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False

if __name__ == "__main__":
    root = tk.Tk()
    app = LineDrawingApp(root)
    root.mainloop()

    # After closing the app, use the coordinates to check if lines intersect
    line1_coords = app.line1_coords
    line2_coords = app.line2_coords

    L1 = line([line1_coords[0][0], line1_coords[0][1]] , [line1_coords[1][0], line1_coords[1][1]])
    L2 = line([line2_coords[0][0], line2_coords[0][1]] , [line2_coords[1][0], line2_coords[1][1]])

    fig, ax = plt.subplots()
    ax.set_xlim(0, 500)
    ax.set_ylim(0, 500)

    R = intersection(L1, L2)        
    if (min(line1_coords[0][0], line1_coords[1][0]) <= R[0] <= max(line1_coords[0][0], line1_coords[1][0]) and
        min(line1_coords[0][1], line1_coords[1][1]) <= R[1] <= max(line1_coords[0][1], line1_coords[1][1]) and
        min(line2_coords[0][0], line2_coords[1][0]) <= R[0] <= max(line2_coords[0][0], line2_coords[1][0]) and
        min(line2_coords[0][1], line2_coords[1][1]) <= R[1] <= max(line2_coords[0][1], line2_coords[1][1])):
            print ("\nIntersection point : ", R)
            ax.plot(R[0], R[1], 'ro', label='Intersection Point')
    elif R == False:
        print("\nNo Intersection")
    else:
        print("\nNo Intersection")

    # Plot lines
    l1, = ax.plot([line1_coords[0][0], line1_coords[1][0]], [line1_coords[0][1], line1_coords[1][1]], label='Line 1')
    l2, = ax.plot([line2_coords[0][0], line2_coords[1][0]], [line2_coords[0][1], line2_coords[1][1]], label='Line 2')

    plt.legend()
    plt.show()

    print(f"Time Complexity  : O({1})")
    print(f"Space Complexity : O({1})")
